<a href="https://colab.research.google.com/github/eengel7/transformer_speech_transcription/blob/main/feature_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


In [ ]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt update
!apt install -y ffmpeg

In [ ]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio
!pip install huggingface

In [ ]:
!pip install huggingface_hub

In [6]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [ ]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "de", split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "de", split="test", use_auth_token=True)


In [ ]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

In [ ]:
print(common_voice,common_voice["train"].info)


In [ ]:
import sys
size = sys.getsizeof(common_voice["train"])
print(size, size/1024, size/(1024**2))

In [17]:
os.getcwd()

'/content'

In [ ]:
common_voice["test"].save_to_disk("drive/MyDrive/CommonVoice/common_voice_test_2nd_version.hf")

In [13]:
common_voice["train"].save_to_disk("drive/MyDrive/CommonVoice/common_voice_training.hf")

KeyboardInterrupt: ignored

In [ ]:
# test to get half the data set 
common_voice["train"][0:247545].save_to_disk("drive/MyDrive/CommonVoice/common_voice_training_first_part.hf")

In [ ]:
# LATER to upload local data
!pip install hopsworks

     |████████████████████████████████| 2.3 MB 43.4 MB/s 
  Created wheel for hopsworks: filename=hopsworks-3.0.4-py3-none-any.whl size=64205 sha256=d12d18cafef16406f318d1498b018c1a202562f5fc5f5e7632d20f05d2200446
  Stored in directory: /root/.cache/pip/wheels/06/6c/29/cc574a7f42e873dccd45d7e7b0dbe4d55b0320925fb7969b4a
  Created wheel for hsfs: filename=hsfs-3.0.5-py3-none-any.whl size=176226 sha256=29afd000140bcb9a948b2b1165272e8d7db39e2c28e1736ec384312c0c0b9d53
  Stored in directory: /root/.cache/pip/wheels/0d/dd/82/9c0962e6557feae899fa816298fd24838551002324db64467f
  Created wheel for avro: filename=avro-1.10.2-py3-none-any.whl size=96832 sha256=6d7e4f88aee23801d1ef1a1bbda533d21514a6734521f95eb5dac8512b5e876d
  Stored in directory: /root/.cache/pip/wheels/66/b5/b3/185a0da0ecbc3e902e24d1e2fa415db0c7342d6e3633c49d30
  Created wheel for hsml: filename=hsml-3.0.3-py3-none-any.whl size=99264 sha256=843301ec8c28ba665851473c08859fb846cb4b8389d9450799fbe032fc91f1e9
  Stored in directory: /r

Copy your Api Key (first register/login): https://c.app.hopsworks.ai/account/api/generated


In [ ]:
import hopsworks
project = hopsworks.login()


dataset_api = project.get_dataset_api()
dataset_api.upload(common_voice, "Resources")